In [ ]:
import pandas as pd

dataset = pd.read_csv('uaScoresDataFrame.csv')

print('First few rows of the dataset:')
print(dataset.head())
print("\nData types of each column:")
print(dataset.dtypes)
print("\nSummary statistics:")
print(dataset.describe())
print("\nMissing values per column:")
print(dataset.isnull().sum())

First few rows of the dataset:
   Unnamed: 0      UA_Name    UA_Country   UA_Continent  Housing  \
0           0       Aarhus       Denmark         Europe   6.1315   
1           1     Adelaide     Australia        Oceania   6.3095   
2           2  Albuquerque    New Mexico  North America   7.2620   
3           3       Almaty    Kazakhstan           Asia   9.2820   
4           4    Amsterdam   Netherlands         Europe   3.0530   

   Cost of Living  Startups  Venture Capital  Travel Connectivity  Commute  \
0           4.015    2.8270            2.512               3.5360  6.31175   
1           4.692    3.1365            2.640               1.7765  5.33625   
2           6.059    3.7720            1.493               1.4555  5.05575   
3           9.333    2.4585            0.000               4.5920  5.87125   
4           3.824    7.9715            6.107               8.3245  6.11850   

   ...  Safety  Healthcare  Education  Environmental Quality  Economy  \
0  ...  9.6165    